In [ ]:
import pandas as pd
import glob
import os
import shutil
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [ ]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [ ]:
dataset_names = ['Dataset011_5FoldCV', 'Dataset012_5FoldCV', 'Dataset013_5FoldCV', 'Dataset014_5FoldCV', 'Dataset015_5FoldCV']

In [ ]:
preds = glob.glob('/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset01*/test*/*.nii.gz')
len(preds)

In [ ]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: x.replace('test_fold1', 'labelsTs'))
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [ ]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_vol_nii_n(i, [1,2,3,4,5], n_jobs=50) + utils.mp_vol_nii_n(j, [1,2,3,4,5], n_jobs=50))

In [ ]:
df = pd.DataFrame(res)

In [ ]:
df.head()

In [ ]:
df.columns = ['prediction', 'gt', 'Vol_left', 'Vol_right', 'Vol_3rd', 'Vol_4th', 'Vol_Acq', 'gt_Vol_left', 'gt_Vol_right', 'gt_Vol_3rd', 'gt_Vol_4th', 'gt_Vol_Acq']


In [ ]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [ ]:
df['sid'] = 'sub-V' + df.sid

In [ ]:
df.head()

In [ ]:
frees_etivs = '/home/orco/data/VentrikelCNN/code/stats/freesurfer_eTIVs.csv'
etivs = pd.read_csv(frees_etivs)

In [ ]:
etivs.head()

In [ ]:
etivs['sid'] = etivs.path.apply(lambda x: x.split('/')[9])
etivs.head()                   

In [ ]:
df = df.merge(etivs, on='sid').drop(columns='path')
df = df[['sid', 'eTIV'] +[i for i in df.columns if i.startswith('Vol') or i.startswith('gt_Vol')]]
df.head()

In [ ]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

In [ ]:
df.to_csv('./data/volumes_nnUNet_GT_5classes_{}_{}.csv'.format('5FoldCV', date), index=False)